In [3]:
import pandas as pd
import requests
import time
import re
import numpy as np
from bs4 import BeautifulSoup
from sklearn.linear_model import LinearRegression

#Get the comment straight from request. Then extract the comment ("<!"). Use BeautifulSoup on this comment.

In [3]:
data = pd.DataFrame(columns= ['Name', 'Year','Wins','Champion','MVP','Scoring Leader', 
                              'Rebound Leader', 'Assist Leader', 'WS Leader','DPOY',
                              'MIP', '6MOY', 'Coach of Year', 'All-Stars', 'All-Defensive',
                              #'All-NBA', 
                              'FG%','3P Attempts', '3P%', 
                              '2P%', '2P Attempts' 'FT%', 'PPG', 'TOVPG', 'APG','RPG', 'STL', 'BLK', 'FG2A', 'FG3A' 
                             ])
data

,Name,Year,Wins,Champion,MVP,Scoring Leader,Rebound Leader,Assist Leader,WS Leader,DPOY,...,2P%,2P AttemptsFT%,PPG,TOVPG,APG,RPG,STL,BLK,FG2A,FG3A


In [4]:
def check_roster(team_soup, season_awards, allst_roster, all_def):
    awards = [0,0,0,0,0,0,0,0,0,0,0]
    players = team_soup.find_all('td', {'class':'left', 'data-stat':'player'})
    roster = [player.text for player in players]
    coach = roster.append(team_soup.find(text=re.compile('Coach:')).parent.parent.find('a').text)
    
    star_count = 0
    all_def_count = 0
    for allst in allst_roster:
        if allst in roster:
            star_count += 1
 
    awards[9] = star_count
    for i in range(9):
        if season_awards[i] in roster:
            awards[i] = 1
            
    for player in roster:
        if player in all_def:
            all_def_count +=1 
    awards[10] = all_def_count
    
    return awards

In [5]:
## Get Team Row
def get_team_row(all_def, season_awards, allst_roster, name, team_url, year, wins):
    row = []
    row.append(name.replace('*',''))
    row.append(year)
    row.append(wins)
    
    roster = []
    
    url = 'http://www.basketball-reference.com' + team_url
    req = requests.get(url)
    time.sleep(0.1)
    team_soup = BeautifulSoup(req.text, 'html.parser')
    try:
        row.append(team_soup.find('br').text[:4] == 'Won ')
    except:
        row.append(False)
    row += check_roster(team_soup, season_awards, allst_roster,all_def)
    
    row += get_statline(req.text)
    print(row)
    
    return row

In [6]:
leaders = ['Most Valuable Player', 'PPG Leader', 'RPG Leader',
           'APG Leader', 'WS Leader']

awards = ['Defensive Player of the Year', 'Most Improved Player', 
          'Sixth Man of the Year', 'Coach of the Year']
winners = []

def get_award_winners(season, year):
    for leader in leaders:
        winners.append(season.body.find(text=leader).parent.parent.find('a').text)

    #Using a different site that's more convenient to scrape other awards.
    year_string = '%s' % str(year)
    next_year = '%s' % str(year+1)
    year = '%s-%s' % (year_string, next_year[2:4])
    for award in awards:
        url = 'http://www.nba.com/history/%s-award/' %(award.lower().replace(' ', '-'))
        req = requests.get(url)
        soup = BeautifulSoup(req.text, 'html.parser')

        table = soup.find('table')    
        try:
            winners.append(table.find(text=re.compile(year)).parent.parent.find_all('td')[1].text.strip())
        except:
            winners.append('')
    
    return winners

In [15]:
# Append all_stars rosters to list
def get_allstars(year):
    url = "http://www.basketball-reference.com/allstar/NBA_%s.html/" % str(year)
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    all_stars = []
    
    for row in soup.find("div", {"id" : "all_East"}).find_all("th", {"class" : "left", "scope" : "row", "csk" : True}):
        all_stars.append(row.find("a").text)
        
        
    for row in soup.find("div", {"id" : "all_West"}).find_all("th", {"class" : "left", "scope" : "row", "csk" : True}):
        all_stars.append(row.find("a").text)
        
    return all_stars

In [11]:
# Because we need only 1 request to get the whole defensive page, each five players from list is from the First team of a season.
def get_all_def():
    url = "http://www.nba.com/history/all-defensive-teams/"
    req = requests.get(url)
    defensive = BeautifulSoup(req.text, 'html.parser')
    all_defense = []
    
    for i in defensive.find_all("td" , {"class" : "cnnIEColTxtL", "style" : "text-align: center"}):
        if i.text == "First Team":
            continue
        all_defense.append(i.text)
        
    for i in defensive.find_all("td" , {"class" : "cnnIEColTxtR", "style" : "text-align: center"}):
        if i.text == "Second Team":
            continue
        all_defense.append(i.text)
    return all_defense

- get_all_nba() function: scrape data from the http://www.basketball-reference.com/awards/all_league.html. Get all players from the first team, second team, third team and stores them in a dictionary. The function returns a dictionary with keys as years from 1950-2017. Each keys is a list of 3 lists. 

In [22]:
def get_all_nba():
    # Get request
    req = requests.get("http://www.basketball-reference.com/awards/all_league.html")
    soup = BeautifulSoup(req.text, "html.parser")
    
    # Variables to keep tracks of the difficult parts to parse
    j = 0
    i = 0
    data_stat = 1
    
    # Initialize variables
    prev_year = "2017"
    dict_allnba = {

    }
    
    players = []

    # For-loop 
    for row in soup.find_all("tr", {"class" : False}):
        content = row.text
        
        # Parse year from "2016-17" to be in normal format "2017"
        year = content[:7]
        year = year[:2] + year[-2:]
        
        # Get the league name
        league = content[7:10]

        if league == "NBA":
            if ((i == 2) & (year != prev_year)):
                dict_allnba[prev_year].append(players)
                
                data_stat = 1
                i = 0
                j += 1

            if (i == 3):
                i = 0
                data_stat = 1


            # If the row is none, skip it         
            if (soup.find_all("td", {"class" : "left", "data-stat" : "%s" % str(1)})[j].find("a") == None):
                j += 1

            # Create a list of players to hold 5 players
            players = []   
            
            # Initialize key
            if year not in dict_allnba:
                dict_allnba[year] = []
            
            # Get 5 players each team
            for stat in range(data_stat, data_stat + 5):
                
                try:
                    # Append players from second team and third team
                    if (stat > 5) & (j != 0): 
                        new_j = int(j / 2)

                        players.append(soup.find_all("td", {"class" : "left", "data-stat" : "%s" % str(stat)})[new_j].find("a").text)
                    else:
                    # Append players from first team
                        players.append(soup.find_all("td", {"class" : "left", "data-stat" : "%s" % str(stat)})[j].find("a").text)
                except:
                    pass           
                
                # Append the team to dictionary
                if (stat == 5) | (stat == 10) | (stat == 15):
                    dict_allnba[year].append(players)
                    players = []

            # Increament variables
            data_stat += 5
            i += 1
            prev_year = year

            if (data_stat == 16):
                data_stat = 1
                j += 1
        
        elif (league == "ABA") | (league == "BAA") :
            j += 1
    
    dict_allnba["2000"] = dict_allnba.pop("1900")
    return dict_allnba

In [23]:
def get_statline(page_text):
    stats = [0,0,0,0,0,0,0,0,0,0,0,0,0]
    stat_search = ['fg_pct', 'fg3a', 'fg3_pct', 'fg2a', 'fg2_pct',
                   'ft_pct','trb_per_g','ast_per_g','stl_per_g',
                   'blk_per_g','tov_per_g','pts_per_g']
    
    for i in range(len(stat_search)):
        indexer = page_text.find('data-stat="%s" >' % (stat_search[i]))
        try:
            offset = len(stat_search[i]) + 14
            stat = re.sub("<", '',page_text[indexer+offset: indexer+offset+4])    
            stat = float(stat)
        except:
            stat = 0
        stats[i] = stat
    return stats

In [ ]:
row_num = 0
#all_def = get_all_def()
#all_nba = get_all_nba()
for i in range(1960,2018):
    print(i)
    double_count = 0
    url = ('http://www.basketball-reference.com/leagues/NBA_%s.html' % str(i))
    time.sleep(0.1)
    req = requests.get(url)
    season = BeautifulSoup(req.text, 'html.parser')
    teams = season.find_all('th', {'class': 'left', 'data-stat':'team_name'})
   
    season_awards = get_award_winners(season, i)
    allst_roster = get_allstars(i)
    all_nba_team = all_nba[str(i)]
    
    for team in teams:
        double_count += 1
        if(double_count > 32):
            break
        if(team.text == 'Team' or 'Conference' in team.text):
            continue
        wins = int(team.parent.find('td', {'data-stat':'wins'}).text)
     
        team_name = repr(team.text).replace('\\x','*').split('*')[0].strip("'")
        for a in team.find_all('a',href=True):
            new_row = get_team_row(all_def,all_nba_team,season_awards,allst_roster,team_name,a['href'], i, wins)
            data.loc[row_num] = new_row
            row_num += 1


In [ ]:
name_changes = [['Rockets', 'Houston Rockets'], ['Lakers', 'Los Angeles Lakers'], ['Jazz', 'Utah Jazz'],
                ['Hawks', 'Atlanta Hawks'], ['Clippers', 'Los Angeles Clippers'], ['Grizzlies', 'Memphis Grizzlies'],
                ['SuperSonics', 'Oklahoma City Thunder'], ['Kings', 'Sacramento Kings'], 
                ['Royals', 'Sacramento Kings'], ['Zephyrs', 'Washington Wizards'],
                ['Bobcats', 'Charlotte Hornets'], ['Warriors', 'Golden State Warriors'],
                ['New Orleans', 'New Orleans Pelicans'], ['Syracuse', 'Philadelphia 76ers'],
                ['Nets','Brooklyn Nets'], ['Bullets', 'Washington Wizards'], 
                ['Buffalo', 'Los Angeles Clippers'], ['Packers', 'Washington Wizards'] ]
for name_change in name_changes:
    data.ix[data.Team.str.contains(name_change[0]), 'Team'] = name_change[1]



In [ ]:
data['Accolades'] = (data['MVP'] + data['Scoring Leader'] + data['Rebound Leader'] + data['Assist Leader']
                     + data['WS Leader'] + data['DPOY'] + data['MIP'] + data['6MOY'] + data['Coach of Year']
                     + data['All-Stars'] + data['All-Defensive'] + data['All-NBA'])

In [ ]:
data

In [ ]:
data.to_csv('nba_data.csv')